# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config

sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# import tracking
from sort.sort import Sort
import glob

# %matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "pretrained_models", "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# indicate GPUs
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

P2: Tensor("fpn_p2/BiasAdd:0", shape=(?, ?, ?, 256), dtype=float32)
P3: Tensor("fpn_p3/BiasAdd:0", shape=(?, ?, ?, 256), dtype=float32)
rpn_class_logits Tensor("rpn_class_logits/concat:0", shape=(?, ?, 2), dtype=float32)
pooled from pyramid ROI align:  (1, ?, 7, 7, 256)
shared features:  (?, 1000, 1024)
mrcnn_class:  (?, 1000, 81)
rpn_rois:  (1, ?, 4)
shared_features:  (?, 1000, 1024)
rois passed to refine:  (1, ?, 4)
rois in refine:  (?, 4)
refined_rois used for gather keep:  (1000, 4)
keep:  (?,)
after gather:  (?, 4)
shared feature in refined: Tensor("mrcnn_detection/strided_slice_12:0", shape=(1000, 1024), dtype=float32)
detections_batch passed from batch_slice:  (1, ?, 6)
features passed from batch_slice:  (1, ?, 1024)
keep passed to outside is:  (1, ?, 1024)
pooled from pyramid ROI align:  (1, ?, 14, 14, 256)


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench']#, 'bird',
#                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
#                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
#                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
#                'kite', 'baseball bat', 'baseball glove', 'skateboard',
#                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
#                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
#                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
#                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
#                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
#                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
#                'teddy bear', 'hair drier', 'toothbrush']

num_classes = len(class_names)
num_classes

15

## Run Object Detection on Taiwan SA dataset

In [5]:
'''Run on KITTI raw images!'''
# del sys.modules['sort']
# del sys.modules['mrcnn']
from mrcnn import visualize
from sort.sort import Sort
from utils import *
import glob
import time
import numpy as np
from PIL import Image

import os

# only for testing
IMG_DIR = '/media/DATA/VAD_datasets/taiwan_sa/testing/frames/'#'/media/DATA/traffic_accident_videos/images_10hz/'
OUT_DIR = '/media/DATA/VAD_datasets/taiwan_sa/testing/mask_rcnn_detections/'#'/media/DATA/traffic_accident_videos/mask_rcnn_detections/'
SAMPLE_IMG_DIR = OUT_DIR

all_folders = glob.glob(IMG_DIR + '*')

W = 1280
H = 720
ROI = [0, 0, 720, 1280]
saver = True
display = False
    
colors = np.random.rand(32, 3)
'''for saving observations of each video'''
all_observations = {}
for folder in all_folders:
    video_name = folder.split('/')[-1]
    print(video_name)    
    '''for display'''
    if display:
        colours = np.random.rand(32, 3)*255  # used only for display
        plt.ion()
        fig = plt.figure()
    
    '''init tracker'''
    use_dlibTracker = False # True to use dlib correlation tracker, False to use Kalman Filter tracker
#     all_trackers =  Sort(ROI, max_age=3,min_hits=3, use_dlib=use_dlibTracker,track_masks=True)
    all_trackers =  Sort(max_age=3,min_hits=3)
    
    '''count time'''
    total_time = 0

#     '''write results'''
    out_file = os.path.join(OUT_DIR, video_name + '.txt')
    
    feature_out = os.path.join(OUT_DIR, video_name + '.npy')
    
    try:
        os.stat(OUT_DIR)
        print("video has been processed!")
#         continue
    except:
        os.mkdir(OUT_DIR)
        aa = 1
    f_out = open(out_file, 'w')
    frame = 0
    
    '''for saving observations of each car'''
    observations = {}
    tracked_features = []
    all_images = sorted(glob.glob(os.path.join(folder, '*.jpg')))
    for image_file in all_images:
        img = np.asarray(Image.open(image_file))
        
        if img is None:
            break
        
        mrcnn_detections  = model.detect([img], verbose=1)[0]
        interesting_objects = np.where(mrcnn_detections['class_ids']<num_classes)[0]
        
        bbox_hash = {}
        bboxes = mrcnn_detections['rois'][interesting_objects]
        masks = mrcnn_detections['masks'][:,:,interesting_objects]
        classes = mrcnn_detections['class_ids'][interesting_objects]
        scores = mrcnn_detections['scores'][interesting_objects]
        features = mrcnn_detections['roi_features'][interesting_objects]
        
        for i, bbox in enumerate(bboxes):
            bbox_hash[tuple(bbox)] = [classes[i], scores[i],masks[i]]
        
        
        start_time = time.time()
        #update tracker
#         trackers, feature_list = all_trackers.update(bboxes.astype('int'),
#                                                   img=img, 
#                                                   masks=masks,
#                                                   classes=classes,
#                                                   scores=scores,
#                                                   features=features) # only cars are considered so far.
        matched, ret = all_trackers.update(bboxes.astype('int')) # only cars are considered so far.
        if frame == 0:
            matched = [[row[-1],row[-1]] for row in ret]
        print(matched)
        trk2det_id_hash = {}
        for m in matched:
            trk2det_id_hash[m[1]] = m[0]
        
        # use correlation tracker
#         trackers, mask_list = all_trackers.update(bboxes.astype('int'),
#                                        img=img,
#                                        classes=classes,
#                                        scores=scores) # only cars are considered so far.
        
        

        cycle_time = time.time() - start_time
        total_time += cycle_time

        print('frame: %d...took: %3fs'%(frame,cycle_time))
        
        tracked_boxes = []
        tracked_id = []
        tracked_masks = []
        tracked_classes = []
        tracked_scores = []
        
#         if not use_dlibTracker:
#             for j, (d,features) in enumerate(zip(trackers, feature_list)):
#                 tracked_boxes.append(d[:4])
#                 tracked_id.append(d[4])
#                 tracked_classes.append(d[-1])
#                 tracked_scores.append(d[-2])
#                 tracked_features.append(feature_list[j])

#                 # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#                 f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                             (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
                
                
                
#         else:
#             for j, d in enumerate(trackers):
#                 tracked_boxes.append(d[:4])
#                 tracked_id.append(d[4])
#                 tracked_classes.append(d[-1])
#                 tracked_scores.append(d[-2])
#                 tracked_features.append(feature_list[j])
#                 # track_id, frame_id, age, class, score, xmin, ymin, xmax, ymax
#                 f_out.write('%d,%d,%d,%d,%d,%.3f,%.3f,%.3f,%.3f\n' % 
#                             (d[4], frame, d[5], d[7],d[6], d[0], d[1], d[2], d[3]))
        for j, d in enumerate(ret):
            tracked_boxes.append(d[:4])
            tracked_id.append(d[4])
            tracked_classes.append(classes[trk2det_id_hash[d[4]]])
            tracked_scores.append(scores[trk2det_id_hash[d[4]]])
            tracked_masks.append(masks[trk2det_id_hash[d[4]]])
            
            f_out.write('%d,%d,%d,%.3f,%.3f,%.3f,%.3f,%.3f\n' % 
                            (d[4], frame,tracked_classes[j], tracked_scores[j], d[0], d[1], d[2], d[3]))
            
        tracked_boxes = np.array(tracked_boxes).astype('int')
        tracked_id = np.array(tracked_id)
        if len(tracked_id) == 0:
            continue
        tracked_classes = np.array(tracked_classes).astype('int')
        tracked_scores = np.array(tracked_scores)

    
#         save masked images
        save_path = os.path.join(SAMPLE_IMG_DIR, video_name, str(format(frame,'04'))+'.jpg')
        masked_img = visualize.display_tracklets(img,
                                            tracked_boxes,
                                            tracked_id,
                                            tracked_masks, 
                                            tracked_classes, 
                                            class_names, 
                                            tracked_scores,
                                            show_mask = False,
                                            colors = colors,
                                            save_path = save_path)  # used only for display)
        if frame > 5:
            break
        frame += 1
#     tracked_features = np.array(tracked_features)
#     print(tracked_features.shape)
#     np.save(feature_out, tracked_features)
    
    
#         plt.clf()
        
#         save mask files
#         total_mask = np.zeros((640,1280),dtype=bool)
#         for i in range(tracked_masks.shape[2]):
#             total_mask = np.bitwise_or(total_mask, tracked_masks[:,:,i])

#         bbox_mask = np.ones((640,1280))
#         for box in tracked_boxes:
#             bbox_mask[box[0]:box[2], box[1]:box[3]] = 0
#         write_csv(out_path + str(format(frame,'04')) + '.csv' ,total_mask)
    
    print("One video is written!")
    f_out.close()
    break

000456
video has been processed!
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


/home/brianyao/Envs/tf/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


detections (1, 100, 6)
features:  (1, 4, 1024)
[[3, 3], [2, 2], [1, 1], [0, 0]]
frame: 0...took: 0.000542s
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
detections (1, 100, 6)
features:  (1, 4, 1024)
[[0 0]
 [1 1]
 [2 2]
 [3 3]]
frame: 1...took: 0.094417s
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.2

/home/brianyao/Envs/tf/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


detections (1, 100, 6)
features:  (1, 4, 1024)
[[0 0]
 [1 2]
 [2 1]
 [3 3]]
frame: 2...took: 0.000933s
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
detections (1, 100, 6)
features:  (1, 4, 1024)
[[0 0]
 [1 1]
 [2 2]
 [3 3]]
frame: 3...took: 0.000985s
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134

In [6]:
ret

array([[374, 701, 404, 737,   3],
       [359, 523, 519, 689,   2],
       [289, 218, 506, 536,   1],
       [382, 671, 433, 716,   0]])

In [7]:
classes

array([3, 3, 8, 3], dtype=int32)

In [8]:
bboxes.astype('int')

array([[383, 671, 433, 717],
       [362, 523, 518, 689],
       [290, 215, 506, 539],
       [375, 701, 404, 738]])

In [9]:
matched

array([[0, 0],
       [1, 2],
       [2, 1],
       [3, 3]])

In [10]:
# import csv
# import cv2
# import numpy as np
# def read_binary(file_path,header=True,delimiter=','):
#     # The read-in data should be a N*W matrix,
#     # where N is the length of the time sequences,
#     # W is the number of sensors/data features
#     i = 0
#     with open(file_path, 'r') as file:
#         reader = csv.reader(file, delimiter = delimiter)
#         data=[]
#         for line in reader:
#             if i == 0 and header:
#                 i += +1
#             else:
#                 for j, element in enumerate(line):
#                     if element == 'True':
#                         line[j] = 0
#                     elif element == 'False':
#                         line[j] = 255
#                     else:
#                         raise ValueError("Data type is not boolean!!")
                    
#                 line = np.array(line) # str2float
#                 if i == 0  or (i == 1 and header):
#                     data = line
#                 else:
#                     data = np.vstack((data, line))
#                 i += 1
#     return data

In [11]:
# # from data_reader import *
# masks = read_binary('/home/yyao/Documents/car_intersection/tracking_output/mask_rcnn/201804171444003136/0001.csv',header=False)